In [3]:
import os
import tqdm
import pandas as pd
from pymongo import MongoClient

In [4]:
db = MongoClient("mongodb://localhost:27017")['tree2']
predation = db.predation
summary = db.summary

In [5]:
data_path = '../data/20241229-compressed'

In [6]:
predation_files = []
summary_files = []

for root, dirs, files in os.walk(data_path):
    for file in files:
        if 'predation' in file.lower():
            predation_files.append(os.path.join(root, file))
        if 'summary' in file.lower():
            summary_files.append(os.path.join(root, file))

In [11]:
def read_and_save_file(file):
    if file.endswith('.csv'):
        df = pd.read_csv(file)
        if 'ID' in df.columns:
            df.rename(columns={'ID': '_id'}, inplace=True)
        if 'predation' in file.lower():
            predation.insert_many(df.to_dict('records'))
        elif 'summary' in file.lower():
            summary.insert_many(df.to_dict('records'))
        else:
            raise ValueError(f"Unsupported file class: {file}")
    else:
        raise ValueError(f"Unsupported file format: {file}")

In [12]:
for file in tqdm.tqdm(predation_files + summary_files):
    read_and_save_file(file)

100%|██████████| 32/32 [27:24<00:00, 51.38s/it]  
